# Importing Libraries

In [253]:
import sys
import pandas as pd
import numpy as np
import plotly
import math
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo
from pmdarima import auto_arima
from sklearn.model_selection import train_test_split
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings
warnings.filterwarnings("ignore")
# To import the main.py file
sys.path.append('../')
from python_files import main

In [254]:
# Setting up plotly to work offline and in jupyter notebooks
pyo.init_notebook_mode(connected = True)
%matplotlib inline

# Data Preprocessing Functions

In [255]:
def get_data():
    confirmed_global, deaths_global, recovered_global, country_cases = main.collect_data()
    recovered = recovered_global.groupby("country").sum().T
    deaths = deaths_global.groupby("country").sum().T
    confirmed = confirmed_global.groupby("country").sum().T
    deaths.index = pd.to_datetime(deaths.index, infer_datetime_format = True)
    recovered.index = pd.to_datetime(recovered.index, infer_datetime_format = True)
    confirmed.index = pd.to_datetime(confirmed.index, infer_datetime_format = True)
    return deaths, recovered, confirmed

In [256]:
def create_data_frame(dataframe,country):
    deaths, recovered, confirmed = get_data()
    if dataframe == 'deaths':
        data = pd.DataFrame(index = deaths.index, data = deaths[country].values, columns = ["Total"])

    elif dataframe == 'recovered':
        data = pd.DataFrame(index = recovered.index, data = recovered[country].values, columns = ["Total"])

    elif dataframe == 'confirmed':
        data = pd.DataFrame(index = confirmed.index, data = confirmed[country].values, columns = ["Total"])

    data = data[(data != 0).all(1)]
    
    data['Date'] = data.index
    cols = [data.columns[-1]] + [col for col in data if col != data.columns[-1]]
    data = data[cols]   

    return data

# Graphing Functions

In [257]:
def plot_forecast(data,forecast):
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=data["Date"], y=data["Total"],   
                            mode='lines',
                            name='Up till now '))
        
    fig.add_trace(go.Scatter(x=forecast.index, y=forecast.values,   
                            mode='lines',
                            name='prediction'))

    fig.update_layout(title={
                'text': "Forecasted results",
                'y':0.9,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
                        template = "plotly_dark",
                        xaxis_title="Date",
                        yaxis_title="Cases",
                        legend_title="Legend ",
                        font=dict(
                                family="Arial",
                                size=15,
                                color="white"
                                )
                        )
    return fig

# Functions to Train the Model

In [258]:
def find_params(train_set):
    stepwise_model = auto_arima(train_set,method='nm',start_p = 0, start_q = 0,
                               max_p = 2, max_q = 2, m = 7,
                               start_P = 0,max_P=0,start_Q=1,max_Q=1,seasonal = True,
                               d = None, D = 1, n_jobs=-1,trace = True,
                               error_action = 'ignore',  
                               suppress_warnings = True, 
                               stepwise = True)
    return stepwise_model

In [259]:
def Predict(stepwise_model,train,test):
    
    stepwise_model.fit(train)
    
    pred = stepwise_model.predict(n_periods=len(test))
    
    pred = pd.DataFrame(pred,index = test.index,columns=['Prediction'])
   
    return pred

# Error Functions

In [260]:
def mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Functions for Forecasting

In [261]:
def Future(df,order,seasonal_order,train,test,data):
    
    model = SARIMAX(df['Total'],  
                        order = order,  
                        seasonal_order = seasonal_order) 
    result = model.fit() 
  
    forecast = result.predict(start = len(df),  
                          end = (len(df)-1) + 14).rename('Forecast') 
    
 
    error_check = result.predict(start = len(train), end = len(train) - 1 +len(test))
    error = mape(error_check,test)
    error = error
    graph = plot_forecast(data,forecast)

    return forecast,graph,error

# Calling Function

In [262]:
def arima_predict(df_name,country):
    data = create_data_frame(df_name,country)
    train = data["Total"][:len(data)*4//5]
    test = data["Total"][len(data)*4//5:]
    model = find_params(train)
    pred = Predict(model,train,test)
    mape_error = mape(test, pred["Prediction"])
    order=model.get_params()['order']
    seasonal_order=model.get_params()['seasonal_order']
    pred=Predict(model,train,test)
    forecast,graph,error = Future(data,order, seasonal_order, train, test,data)
    return forecast,graph,np.std([error, mape_error])



# Example

In [267]:
forecast,graph,error= arima_predict("confirmed","Russia")

Performing stepwise search to minimize aic
Fit ARIMA(0,2,0)x(0,1,1,7) [intercept=True]; AIC=2525.991, BIC=2535.381, Time=0.183 seconds
Fit ARIMA(0,2,0)x(0,1,0,7) [intercept=True]; AIC=2619.101, BIC=2625.360, Time=0.078 seconds
Fit ARIMA(1,2,0)x(0,1,0,7) [intercept=True]; AIC=2600.171, BIC=2609.560, Time=0.093 seconds
Fit ARIMA(0,2,1)x(0,1,1,7) [intercept=True]; AIC=2526.067, BIC=2538.587, Time=0.212 seconds
Fit ARIMA(0,2,0)x(0,1,0,7) [intercept=False]; AIC=2617.103, BIC=2620.233, Time=0.026 seconds
Fit ARIMA(1,2,0)x(0,1,1,7) [intercept=True]; AIC=2525.631, BIC=2538.151, Time=0.175 seconds
Fit ARIMA(2,2,0)x(0,1,1,7) [intercept=True]; AIC=2526.968, BIC=2542.617, Time=0.246 seconds
Fit ARIMA(1,2,1)x(0,1,1,7) [intercept=True]; AIC=2526.862, BIC=2542.512, Time=0.200 seconds
Fit ARIMA(2,2,1)x(0,1,1,7) [intercept=True]; AIC=2532.714, BIC=2551.494, Time=0.203 seconds
Near non-invertible roots for order (2, 2, 1)(0, 1, 1, 7); setting score to inf (at least one inverse root too close to the bord

In [268]:
print("PREDICTIONS FOR THE NEXT 14 DAYS : \n")
for i in range(len(forecast)):
    print(forecast.index[i] , "---->", int(forecast.values[i]))
  

PREDICTIONS FOR THE NEXT 14 DAYS : 

2020-09-10 00:00:00 ----> 1042761
2020-09-11 00:00:00 ----> 1048105
2020-09-12 00:00:00 ----> 1053505
2020-09-13 00:00:00 ----> 1059004
2020-09-14 00:00:00 ----> 1064395
2020-09-15 00:00:00 ----> 1069766
2020-09-16 00:00:00 ----> 1075075
2020-09-17 00:00:00 ----> 1080482
2020-09-18 00:00:00 ----> 1085993
2020-09-19 00:00:00 ----> 1091560
2020-09-20 00:00:00 ----> 1097226
2020-09-21 00:00:00 ----> 1102784
2020-09-22 00:00:00 ----> 1108322
2020-09-23 00:00:00 ----> 1113799


In [269]:
graph.show()

In [272]:
print("ALLOW AN ERROR OF UPTO (Based on the data at hand) :",error,"%")

ALLOW AN ERROR OF UPTO (Based on the data at hand) : 0.3919207676738036 %
